In [ ]:
import lightningchart as lc
import pandas as pd

lc.set_license('my-license-key')

file_path = "../Dataset/preprocessed_trendedpointalarm1.csv"
df = pd.read_csv(file_path)

In [ ]:
print(df.head(20))

In [ ]:
print(df.describe())

In [ ]:
import lightningchart as lc
import pandas as pd
import numpy as np


# Convert DateTime to datetime format and clean AlarmSeverityName
df["DateTime"] = pd.to_datetime(df["DateTime"], dayfirst=True, errors='coerce')
df["AlarmSeverityName"] = df["AlarmSeverityName"].astype(str).str.split(" - ").str[-1]

# We want a dashboard with 10 columns and 9 rows.
dashboard = lc.Dashboard(columns=10, rows=9, theme=lc.Themes.Dark)

# 1. Pie Chart (Alarm Severity Distribution)
pie_chart = dashboard.PieChart(
    column_index=0, row_index=0, column_span=2, row_span=3
)
pie_chart.set_title("Alarm Severity Distribution")
pie_chart.set_label_formatter("NamePlusValue")
# Compute counts per severity
severity_counts = df["AlarmSeverityName"].value_counts()
slices = []
for severity in severity_counts.index:
    slices.append({
        'name': severity,
        'value': int(severity_counts[severity])
    })
pie_chart.add_slices(slices)
pie_chart.set_inner_radius(50)
pie_chart.add_legend(data=pie_chart).set_title("Severity Levels")

# 2. Stacked Bar Chart – Alarm Class Distribution by Severity
bar_chart = dashboard.BarChart(
    column_index=0, row_index=3, column_span=2, row_span=3
)
bar_chart.set_title("Alarm Class Distribution by Severity")
# Group data by AlarmClassName and AlarmSeverityName
class_severity_counts = df.groupby(["AlarmClassName", "AlarmSeverityName"]).size().unstack().fillna(0)
categories = list(class_severity_counts.index)
severity_labels = list(class_severity_counts.columns)
data = []
for severity in severity_labels:
    data.append({"subCategory": severity, "values": list(class_severity_counts[severity])})
bar_chart.set_data_stacked(categories, data)
bar_chart.set_value_label_display_mode("hidden")
bar_chart.add_legend().add(bar_chart)

# 3. Horizontal Bar Chart – Top 10 Asset IDs with Most Alarms
bar_chart_asset = dashboard.BarChart(
    column_index=0, row_index=6, column_span=2, row_span=3,
)
bar_chart_asset.set_title("Top 10 Asset IDs with Most Alarms")
bar_chart_asset.set_sorting("disabled")
# Count alarms per AssetID and select top 10
asset_counts = df["AssetID"].value_counts()
top_assets = asset_counts[:10]
asset_data = [
    {"category": str(top_assets.index[i]), "value": int(top_assets.values[i])}
    for i in range(len(top_assets))
]
bar_chart_asset.set_data(asset_data)
# Configure palette based on asset counts
min_value = int(min(top_assets.values))
max_value = int(max(top_assets.values))
bar_chart_asset.set_palette_colors(
    steps=[
        {"value": min_value, "color": lc.Color("blue")},
        {"value": max_value * 0.25, "color": lc.Color("cyan")},
        {"value": max_value * 0.5, "color": lc.Color("green")},
        {"value": max_value * 0.75, "color": lc.Color("yellow")},
        {"value": max_value, "color": lc.Color("red")},
    ],
    look_up_property="y",
    percentage_values=False,
)


# Group B (Middle panel – next 4 columns: columns 2-5)

# 4. Heatmap of Alarm Counts by Day & Hour
# Prepare the heatmap data
df["Day"] = df["DateTime"].dt.day
df["Hour"] = df["DateTime"].dt.hour
heatmap_data = df.pivot_table(
    index="Day", columns="Hour", values="ProcessID", aggfunc="count", fill_value=0
)
heatmap_array = heatmap_data.to_numpy()
days = heatmap_data.index.tolist()
hours = heatmap_data.columns.tolist()

heatmap_chart = dashboard.ChartXY(
    column_index=2, row_index=0, column_span=4, row_span=5
)
heatmap_chart.set_title("Heatmap of Alarm Counts by Day & Hour")
heatmap_series = heatmap_chart.add_heatmap_grid_series(
    columns=heatmap_array.shape[1],
    rows=heatmap_array.shape[0],
    data_order='rows'
)
heatmap_series.set_start(x=0, y=0)
heatmap_series.set_end(x=heatmap_array.shape[1], y=heatmap_array.shape[0])
heatmap_series.set_step(x=1, y=1)
heatmap_series.set_wireframe_stroke(thickness=1, color=lc.Color("white"))
heatmap_series.invalidate_intensity_values(heatmap_array.astype(float).tolist())
min_val = float(np.min(heatmap_array))
max_val = float(np.max(heatmap_array))
mid_val = (min_val + max_val) / 2
palette_steps = [
    {"value": min_val, "color": lc.Color("blue")},
    {"value": mid_val, "color": lc.Color("white")},
    {"value": max_val, "color": lc.Color("red")},
]
heatmap_series.set_palette_coloring(
    steps=palette_steps,
    look_up_property="value",
    interpolate=True
)
# Customize axes ticks
x_axis = heatmap_chart.get_default_x_axis().set_tick_strategy("Empty")
y_axis = heatmap_chart.get_default_y_axis().set_tick_strategy("Empty")
for i, hour_val in enumerate(hours):
    tick = x_axis.add_custom_tick().set_tick_label_rotation(0)
    tick.set_value(i + 0.5)
    tick.set_text(str(hour_val))
for j, day_val in enumerate(days):
    tick = y_axis.add_custom_tick()
    tick.set_value(j + 0.5)
    tick.set_text(str(day_val))
x_axis.set_title("Hour of the Day")
y_axis.set_title("Day of the Month")
heatmap_chart.add_legend(data=heatmap_series).set_margin(-20)

# 5. Bubble Chart of Alarm Severity Over Time
df["DateTime"] = pd.to_datetime(df["DateTime"], infer_datetime_format=True, errors="coerce")
df.dropna(subset=["DateTime"], inplace=True)

# Use the cleaned AlarmSeverityName values (e.g., "High", "Medium", "Low")
severity_mapping = {"High": 3, "Medium": 2, "Low": 1}
df["SeverityNumeric"] = df["AlarmSeverityName"].map(severity_mapping)

# Convert DateTime to milliseconds since the Unix epoch
df["TimestampMs"] = (df["DateTime"].astype("int64") // 10**6)

# Group by TimestampMs and SeverityNumeric, counting alarms per group
df_grouped = df.groupby(["TimestampMs", "SeverityNumeric"]).size().reset_index(name="AlarmCount")

# Assuming 'dashboard' is already created, create a ChartXY for the bubble chart:
bubble_chart = dashboard.ChartXY(
    column_index=2, row_index=5, column_span=4, row_span=4
)
bubble_chart.set_title("Alarm Severity Over Time")

# Create three point series for the three severity levels.
series_high = bubble_chart.add_point_series(sizes=True).set_point_color(lc.Color(255, 0, 0, 128))     # Red
series_medium = bubble_chart.add_point_series(sizes=True).set_point_color(lc.Color(255, 255, 0, 128)) # Yellow
series_low = bubble_chart.add_point_series(sizes=True).set_point_color(lc.Color(0, 255, 255, 128))      # Blue

# Append data to each series.
for severity_val, series in [(3, series_high), (2, series_medium), (1, series_low)]:
    subset = df_grouped[df_grouped["SeverityNumeric"] == severity_val]
    series.append_samples(
        x_values=subset["TimestampMs"].tolist(),
        y_values=subset["SeverityNumeric"].tolist(),
        sizes=(subset["AlarmCount"] * 5).tolist(),  # Adjust scaling as needed
    )

# Configure axes.
x_axis = bubble_chart.get_default_x_axis()
x_axis.set_title("Date/Time")
x_axis.set_tick_strategy("DateTime")

y_axis = bubble_chart.get_default_y_axis()
y_axis.set_title("Severity Level (1=Low, 2=Medium, 3=High)").set_interval(0.9, 3.3, stop_axis_after=True)


# 6. Alarm Severity Changes Over Time (Stacked Area Chart - Log Scale)
# Prepare daily aggregation by severity
df["Date"] = df["DateTime"].dt.date
severity_counts = df.groupby(["Date", "AlarmSeverityName"]).size().unstack(fill_value=0)
# Ensure the expected columns exist and order them
for sev in ["Low", "Medium", "High"]:
    if sev not in severity_counts.columns:
        severity_counts[sev] = 0
severity_counts = severity_counts[["Low", "Medium", "High"]]
severity_counts["Timestamp"] = pd.to_datetime(severity_counts.index).astype('int64') // 10**9
severity_counts = severity_counts.sort_values("Timestamp")
x_values = severity_counts["Timestamp"].tolist()
# Replace zeros for log scaling (using 1 as the minimum)
low = np.maximum(severity_counts["Low"].astype(float), 1).tolist()
medium = np.maximum(severity_counts["Medium"].astype(float), 1).tolist()
high = np.maximum(severity_counts["High"].astype(float), 1).tolist()
stacked_medium = np.add(low, medium).tolist()
stacked_high = np.add(stacked_medium, high).tolist()
low_log = np.log10(low)
medium_log = np.log10(stacked_medium)
high_log = np.log10(stacked_high)

area_chart = dashboard.ChartXY(
    column_index=6, row_index=0, column_span=4, row_span=4
)
area_chart.set_title("Alarm Severity Changes Over Time (Log Scale)")
low_series = area_chart.add_area_series().set_name("Low")
low_series.add(x_values, low_log.tolist())
medium_series = area_chart.add_area_series().set_name("Medium")
medium_series.add(x_values, medium_log.tolist())
high_series = area_chart.add_area_series().set_name("High")
high_series.add(x_values, high_log.tolist())
x_axis = area_chart.get_default_x_axis().set_title("Date").set_tick_strategy("Empty")
y_axis = area_chart.get_default_y_axis().set_title("Log(Number of Alarms)")
y_axis.set_interval(0, np.log10(max(stacked_high)))
# Add custom x-axis ticks for even years (adjust the range as needed)
for year in range(2016, 2025):
    timestamp = int(pd.Timestamp(f"{year}-01-01").timestamp())
    tick = x_axis.add_custom_tick()
    tick.set_value(timestamp)
    tick.set_text(str(year))
legend = area_chart.add_legend()
legend.add(low_series)
legend.add(medium_series)
legend.add(high_series)

# 7. 3D Surface – Alarms by Day of the Week & Hour of the Day
df["DayOfWeek"] = df["DateTime"].dt.dayofweek
heatmap_data_3d = df.pivot_table(
    index="DayOfWeek", columns="Hour", aggfunc="size", fill_value=0
)
heatmap_array_3d = heatmap_data_3d.to_numpy()

surface_chart = dashboard.Chart3D(
    column_index=6, row_index=4, column_span=4, row_span=5
)
surface_chart.set_title("Alarms by Day of the Week & Hour of the Day")
grid_rows, grid_cols = heatmap_array_3d.shape
surface_series = surface_chart.add_surface_grid_series(
    columns=grid_cols, rows=grid_rows, data_order="rows"
)
surface_series.set_start(x=0, z=0)
surface_series.set_end(x=24, z=7)
surface_series.set_step(x=24 / grid_cols, z=7 / grid_rows)
surface_series.invalidate_height_map(heatmap_array_3d.astype(float).tolist())
surface_series.hide_wireframe()
data_min = float(np.min(heatmap_array_3d))
data_max = float(np.max(heatmap_array_3d))
data_mid = (data_min + data_max) / 2
palette_steps = [
    {"value": data_min, "color": lc.Color("blue")},
    {"value": data_mid, "color": lc.Color("white")},
    {"value": data_max, "color": lc.Color("red")},
]
surface_series.set_palette_coloring(
    steps=palette_steps, look_up_property="value", percentage_values=False
)
surface_series.invalidate_intensity_values(heatmap_array_3d.astype(float).tolist())
surface_chart.get_default_x_axis().set_title("Hour of the Day")
surface_chart.get_default_y_axis().set_title("Alarm Count")
surface_chart.get_default_z_axis().set_title("Day of the Week")
surface_chart.add_legend(data=surface_series).set_title("Alarm Count").set_dragging_mode("draggable")

dashboard.open(method='browser')


C:\Users\Soroush\AppData\Local\Temp\ipykernel_13632\2648800716.py:139: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["DateTime"] = pd.to_datetime(df["DateTime"], infer_datetime_format=True, errors="coerce")
127.0.0.1 - - [04/Mar/2025 10:06:31] "GET / HTTP/1.1" 200 -
